### Step 1: Select and Explain the Dataset

The "Boston Housing" dataset, a well-known dataset in regression challenges. This dataset contains housing prices along with various features like crime rate, average number of rooms, etc. It's often used to predict housing prices based on these features.

### Step 2: Read Data (1 point)
Using the scikit-learn library, I'll load the Boston Housing dataset into a Pandas DataFrame.

In [18]:
import pandas as pd

# Load the California Housing Prices dataset
data_url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv"
housing_data = pd.read_csv(data_url)

housing_data.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY

In [19]:
housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


### Step 3: Exploratory Data Analysis (EDA) and Numerical Results (3 points)

In [20]:
# Calculate summary statistics
median_income_mean = housing_data['median_income'].mean()
housing_median_age_max = housing_data['housing_median_age'].max()
total_rooms_sum = housing_data['total_rooms'].sum()

median_income_mean, housing_median_age_max, total_rooms_sum

(3.8706710029069766, 52.0, 54402150.0)

In [6]:
### Step 4: Setup Experiment and Model Training (3 points)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Remove the target variable
X = housing_data.drop('median_house_value', axis=1)
y = housing_data['median_house_value']

# Handle categorical variables using one-hot encoding
categorical_features = ['ocean_proximity']
numeric_features = list(X.columns.difference(categorical_features))
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess and train the linear regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['households',
                                                   'housing_median_age',
                                                   'latitude', 'longitude',
                                                   'median_income',
                                                   'population',
                                                   'total_bedrooms',
                                                   'total_rooms']),
                                                 ('cat', OneHotEncoder(),
                                                  ['ocean_proximity'])])),
                ('model', LinearRegression())])

### Step 5: Evaluation (3 points)

In [28]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, mse, r2


(50670.489235655565, 4908290571.346393, 0.6254382675296296)

### Assertions

In [40]:
#step 2
assert len(housing_data) == 20640, "Incorrect number of rows"

#step3
assert round(median_income_mean, 2) == 3.87, "Incorrect mean median_income"
assert housing_median_age_max == 52, "Incorrect max housing_median_age"
assert total_rooms_sum == 54402150, "Incorrect total_rooms sum"

#step4
assert len(X_train) == 16512, "Incorrect number of training samples"
assert len(X_test) == 4128, "Incorrect number of test samples"

#step5
def mycheck(yp,yt,error_percent=10):
    p=error_percent/100
    return yt*(1-p)<yp and yp < yt*(1+p)
assert  mycheck(mae,50670.489), "Incorrect Mean Absolute Error (MAE)"
assert mycheck(mse,4908290571.346), "Incorrect Mean Squared Error (MSE)"
assert mycheck(r2,0.625), "Incorrect R-squared (R2) score"